<a href="https://colab.research.google.com/github/elijahcw-git/Capstone/blob/main/data_cleaner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
import pandas as pd
import numpy as np

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
crime_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/crime.csv')
weather_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/weather.csv')

In [4]:
crime_df['DATE_OCC'] = pd.to_datetime(crime_df['DATE_OCC'])
weather_df['DATE_OCC'] = pd.to_datetime(weather_df['DATE_OCC'])
merged_df = pd.merge(crime_df, weather_df, on='DATE_OCC', how='inner')

In [10]:
# Assuming 'data' is your DataFrame
pd.set_option('display.max_rows', 150)
unique_crime_codes = merged_df[['Crm_Cd', 'Crm_Cd_Desc']].drop_duplicates().reset_index(drop=True)

# Print the unique pairs of Crm_Cd and Crm_Cd_Desc
print(unique_crime_codes)


     Crm_Cd                                        Crm_Cd_Desc
0       900                           VIOLATION OF COURT ORDER
1       740  VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...
2       626                  INTIMATE PARTNER - SIMPLE ASSAULT
3       624                           BATTERY - SIMPLE ASSAULT
4       946                          OTHER MISCELLANEOUS CRIME
5       341  THEFT-GRAND ($950.01 & OVER)EXCPT,GUNS,FOWL,LI...
6       930             CRIMINAL THREATS - NO WEAPON DISPLAYED
7       440                 THEFT PLAIN - PETTY ($950 & UNDER)
8       910                                         KIDNAPPING
9       753                     DISCHARGE FIREARMS/SHOTS FIRED
10      510                                   VEHICLE - STOLEN
11      940                                          EXTORTION
12      210                                            ROBBERY
13      901                     VIOLATION OF RESTRAINING ORDER
14      251                  SHOTS FIRED AT INHABITED D

In [12]:
def categorize_crime(crime):
    """Map specific crimes to broader categories."""
    # Assault and Battery
    if crime in ['INTIMATE PARTNER - SIMPLE ASSAULT', 'BATTERY - SIMPLE ASSAULT',
                 'ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT', 'CHILD ABUSE (PHYSICAL) - AGGRAVATED ASSAULT',
                 'CHILD ABUSE (PHYSICAL) - SIMPLE ASSAULT', 'BATTERY POLICE (SIMPLE)',
                 'INTIMATE PARTNER - AGGRAVATED ASSAULT', 'ASSAULT WITH DEADLY WEAPON ON POLICE OFFICER',
                 'OTHER ASSAULT', 'BATTERY ON A FIREFIGHTER', 'BATTERY WITH SEXUAL CONTACT']:
        return 'Assault and Battery'
    # Theft and Burglary
    elif crime in ['THEFT-GRAND ($950.01 & OVER)EXCPT,GUNS,FOWL,LIVESTK,PROD', 'THEFT PLAIN - PETTY ($950 & UNDER)',
                   'BURGLARY FROM VEHICLE', 'THEFT FROM MOTOR VEHICLE - PETTY ($950 & UNDER)',
                   'THEFT FROM MOTOR VEHICLE - GRAND ($950.01 AND OVER)', 'BURGLARY', 'SHOPLIFTING - PETTY THEFT ($950 & UNDER)',
                   'SHOPLIFTING-GRAND THEFT ($950.01 & OVER)', 'BURGLARY, ATTEMPTED', 'THEFT PLAIN - ATTEMPT',
                   'THEFT FROM MOTOR VEHICLE - ATTEMPT', 'EMBEZZLEMENT, GRAND THEFT ($950.01 & OVER)',
                   'EMBEZZLEMENT, PETTY THEFT ($950 & UNDER)', 'THEFT, PERSON', 'PURSE SNATCHING', 'GRAND THEFT / AUTO REPAIR']:
        return 'Theft and Burglary'
    # Vandalism and Property Damage
    elif crime in ['VANDALISM - FELONY ($400 & OVER, ALL CHURCH VANDALISMS)', 'VANDALISM - MISDEAMEANOR ($399 OR UNDER)']:
        return 'Vandalism and Property Damage'
    # Sexual Offenses
    elif crime in ['RAPE, ATTEMPTED', 'SEXUAL PENETRATION W/FOREIGN OBJECT', 'SODOMY/SEXUAL CONTACT B/W PENIS OF ONE PERS TO ANUS OTH',
                   'RAPE, FORCIBLE', 'SEX,UNLAWFUL(INC MUTUAL CONSENT, PENETRATION W/ FRGN OBJ', 'ORAL COPULATION',
                   'LEWD/LASCIVIOUS ACTS WITH CHILD', 'CHILD PORNOGRAPHY', 'INCEST (SEXUAL ACTS BETWEEN BLOOD RELATIVES)']:
        return 'Sexual Offenses'
    # Robbery and Extortion
    elif crime in ['ROBBERY', 'ATTEMPTED ROBBERY', 'EXTORTION']:
        return 'Robbery and Extortion'
    # Violations and Restraining Orders
    elif crime in ['VIOLATION OF COURT ORDER', 'VIOLATION OF RESTRAINING ORDER', 'VIOLATION OF TEMPORARY RESTRAINING ORDER']:
        return 'Violations and Restraining Orders'
    # Fraud and Financial Crimes
    elif crime in ['THEFT OF IDENTITY', 'DOCUMENT FORGERY / STOLEN FELONY', 'COUNTERFEIT',
                   'DEFRAUDING INNKEEPER/THEFT OF SERVICES, $950 & UNDER', 'DEFRAUDING INNKEEPER/THEFT OF SERVICES, OVER $950.01',
                   'FALSE POLICE REPORT', 'DRIVING WITHOUT OWNER CONSENT (DWOC)', 'GRAND THEFT / INSURANCE FRAUD',
                   'CREDIT CARDS, FRAUD USE ($950 & UNDER)', 'CREDIT CARDS, FRAUD USE ($950.01 & OVER)',
                   'DISHONEST EMPLOYEE - GRAND THEFT', 'DISHONEST EMPLOYEE - PETTY THEFT']:
        return 'Fraud and Financial Crimes'
    # Miscellaneous Crimes
    else:
        return 'Miscellaneous Crimes'

# Apply the categorization function to your DataFrame
merged_df['Crime_Category'] = merged_df['Crm_Cd_Desc'].apply(categorize_crime)

# Display the updated DataFrame to verify the new column
print(merged_df[['Crm_Cd_Desc', 'Crime_Category']].head())

                                         Crm_Cd_Desc  \
0                           VIOLATION OF COURT ORDER   
1  VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...   
2                  INTIMATE PARTNER - SIMPLE ASSAULT   
3                           BATTERY - SIMPLE ASSAULT   
4                          OTHER MISCELLANEOUS CRIME   

                      Crime_Category  
0  Violations and Restraining Orders  
1      Vandalism and Property Damage  
2                Assault and Battery  
3                Assault and Battery  
4               Miscellaneous Crimes  


In [13]:
merged_df['Crime_Category'].unique()

array(['Violations and Restraining Orders',
       'Vandalism and Property Damage', 'Assault and Battery',
       'Miscellaneous Crimes', 'Theft and Burglary',
       'Robbery and Extortion', 'Fraud and Financial Crimes',
       'Sexual Offenses'], dtype=object)

In [15]:
merged_df['Crime_Category'] = pd.Categorical(merged_df['Crime_Category'])
merged_df['Crime_Category_Code'] = merged_df['Crime_Category'].cat.codes

# Optional: Create a mapping of numeric codes to categories for reference
category_mapping = dict(enumerate(merged_df['Crime_Category'].cat.categories))

# Print the first few rows to verify the new column
print(merged_df[['Crime_Category', 'Crime_Category_Code']].head())
print("Category Mapping:", category_mapping)

                      Crime_Category  Crime_Category_Code
0  Violations and Restraining Orders                    7
1      Vandalism and Property Damage                    6
2                Assault and Battery                    0
3                Assault and Battery                    0
4               Miscellaneous Crimes                    2
Category Mapping: {0: 'Assault and Battery', 1: 'Fraud and Financial Crimes', 2: 'Miscellaneous Crimes', 3: 'Robbery and Extortion', 4: 'Sexual Offenses', 5: 'Theft and Burglary', 6: 'Vandalism and Property Damage', 7: 'Violations and Restraining Orders'}


In [16]:
# Adjusted mapping to include Hawaiians and Samoans as Pacific Islanders
region_mapping = {
    'A': 'Asian', 'C': 'Asian', 'D': 'Asian', 'F': 'Asian', 'G': 'Asian',
    'J': 'Asian', 'K': 'Asian', 'L': 'Asian', 'V': 'Asian', 'Z': 'Asian',
    'B': 'Black',
    'H': 'Hispanic/Latin/Mexican',
    'I': 'Other',
    'P': 'Other', 'U': 'Other', 'S': 'Other',
    'W': 'White',
    'O': 'Other',
    'X': 'Unknown'
}

# Apply the revised mapping to the DataFrame
merged_df['Region_Ethnic_Origin'] = merged_df['Vict_Descent'].map(region_mapping).fillna('Other')


In [17]:
merged_df = merged_df[~((merged_df['LAT'] == 0) & (merged_df['LON'] == 0))]
merged_df = merged_df[(merged_df['Avg_Windspeed'] <= 80) & (merged_df['Max_Temp'] >= 20) & (merged_df['Vict_Age'] <= 100)]
merged_df['Vict_Descent'].fillna('Unknown', inplace=True)
merged_df['Total_Precipitation'].fillna(merged_df['Total_Precipitation'].median(), inplace=True)
median_age = merged_df[merged_df['Vict_Age'] > 0]['Vict_Age'].median()
merged_df['Vict_Age'] = merged_df['Vict_Age'].apply(lambda x: median_age if x < 0 else x)
merged_df['Vict_Sex'] = np.where(merged_df['Vict_Sex'].isin(['M', 'F']), merged_df['Vict_Sex'], 'X')
merged_df['Weapon_Reported'] = merged_df['Weapon_Used_Cd'].notna().astype(int)

<ipython-input-17-b82a75de10fc>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df['Vict_Descent'].fillna('Unknown', inplace=True)


In [18]:
merged_df.columns

Index(['DR_NO', 'Date_Rptd', 'DATE_OCC', 'TIME_OCC', 'AREA', 'AREA_NAME',
       'Rpt_Dist_No', 'Part_1-2', 'Crm_Cd', 'Crm_Cd_Desc', 'Mocodes',
       'Vict_Age', 'Vict_Sex', 'Vict_Descent', 'Premis_Cd', 'Premis_Desc',
       'Weapon_Used_Cd', 'Weapon_Desc', 'Status', 'Status_Desc', 'Crm_Cd_1',
       'Crm_Cd_2', 'Crm_Cd_3', 'Crm_Cd_4', 'LOCATION', 'Cross_Street', 'LAT',
       'LON', 'Max_Temp', 'Avg_Temp', 'Min_Temp', 'Max_Dewpoint',
       'Avg_Dewpoint', 'Min_Dewpoint', 'Max_Humidity', 'Avg_Humidity',
       'Min_Humidity', 'Max_Windspeed', 'Avg_Windspeed', 'Min_Windspeed',
       'Max_Pressure', 'Avg_Pressure', 'Min_Pressure', 'Total_Precipitation',
       'Crime_Category', 'Crime_Category_Code', 'Region_Ethnic_Origin',
       'Weapon_Reported'],
      dtype='object')

In [8]:
print(merged_df['Vict_Sex'].unique())
print(merged_df['Vict_Descent'].unique())
print(merged_df['Crm_Cd'].unique())
print(merged_df['Vict_Age'].unique())
print(merged_df['AREA'].unique())
print(merged_df['AREA_NAME'].unique())
print(merged_df['Region_Ethnic_Origin'].unique())

['M' 'F' 'X']
['H' 'O' 'B' 'W' 'Unknown' 'A' 'I' 'X' 'F' 'K' 'P' 'V' 'J' 'C' 'U' 'D' 'S'
 'Z' 'G' 'L' '-']
[900 740 626 624 946 341 930 440 910 753 510 940 210 901 251 330 230 310
 420 745 235 237 812 354 813 954 437 956 442 850 649 763 815 331 928 220
 623 888 627 761 122 821 343 520 410 943 121 810 480 350 351 110 932 441
 421 320 860 652 236 756 806 922 668 660 886 648 845 951 345 439 436 664
 662 820 666 950 433 347 933 487 434 755 474 647 443 661 450 670 762 654
 622 949 471 231 625 653 920 948 902 353 438 865 805 840 352 451 470 890
 444 760 250 651 903 924 944 452 870 822 446 942 435 473 349 113 472 882
 880 475 453 485 884 931 814 921 952 830 445 432 926 906 522 905 904 491]
[48.  0. 25. 53. 18. 46. 21. 49. 35. 36. 57. 22. 50. 23.  9. 43. 24. 26.
 29. 34. 28. 17. 14. 44. 51. 39. 31. 20.  5. 82. 38. 33. 45. 60. 40. 15.
 30. 37. 16.  6. 19. 54. 42. 27. 41. 52. 63. 32. 47. 13. 64. 70. 56. 78.
 73. 55. 66.  7. 58. 59. 76. 62. 61.  8. 67. 71. 69.  3. 72.  4. 12. 81.
 96. 65. 74. 11.

In [ ]:
# merged_df.to_csv('/content/sample_data/merged_dataset.csv', index=False)

In [19]:
columns_to_drop = ['Max_Temp', 'Min_Temp', 'Max_Dewpoint', 'Min_Dewpoint', 'Max_Humidity', 'Min_Humidity',
                   'Max_Windspeed', 'Min_Windspeed', 'Max_Pressure', 'Min_Pressure', 'Weapon_Used_Cd', 'Weapon_Desc',
                   'Crm_Cd_1',  'Crm_Cd_2', 'Crm_Cd_3', 'Crm_Cd_4','Mocodes','Cross_Street', 'Premis_Cd', 'Premis_Desc',
                   'Status', 'Status_Desc']
merged_df.drop(columns=columns_to_drop, inplace=True)

In [20]:
df = merged_df['Crm_Cd'].value_counts().nlargest(10).index
df = merged_df[merged_df['Crm_Cd'].isin(df)]

In [21]:
print(df['Vict_Sex'].unique())
print(df['Region_Ethnic_Origin'].unique())
print(df['Crm_Cd'].unique())
print(df['Vict_Age'].unique())
print(df['AREA'].unique())
print(df['AREA_NAME'].unique())

['M' 'F' 'X']
['Other' 'Hispanic/Latin/Mexican' 'Black' 'White' 'Asian' 'Unknown']
[740 626 624 440 510 330 230 310 420 354]
[ 0. 25. 53. 49. 35. 57. 18. 21. 26. 34. 14. 44. 51. 39. 31. 20. 48. 46.
 28. 38. 23. 33. 45. 60. 40. 30. 50. 22. 37. 29. 16. 24. 19. 54. 42. 27.
 17. 41. 52. 63. 43. 13. 36. 64. 70. 47. 56. 32. 73. 55. 66. 58. 59. 76.
 62. 67. 71. 69. 61. 15. 68. 78. 65. 92. 83.  9. 75. 79. 72. 81.  4. 87.
  6. 85.  3. 82. 74.  7.  8. 84. 12. 77.  2. 80. 96. 88. 11. 86. 90.  5.
 99. 91. 10. 95. 93. 89. 98. 94. 97.]
[ 1  2  3  4  5  6  7  8 12  9 11 10 13 15 14 20 17 21 16 19 18]
['Central' 'Rampart' 'Southwest' 'Hollenbeck' 'Harbor' 'Hollywood'
 'Wilshire' 'West LA' '77th Street' 'Van Nuys' 'Northeast' 'West Valley'
 'Newton' 'N Hollywood' 'Pacific' 'Olympic' 'Devonshire' 'Topanga'
 'Foothill' 'Mission' 'Southeast']


In [22]:
df['Vict_Sex'].value_counts()

M    775974
F    716259
X    325649
Name: Vict_Sex, dtype: int64

In [ ]:
# df_top_10.to_csv('/content/sample_data/df_top_10.csv', index=False)

In [23]:
df['Region_Ethnic_Origin'].value_counts()

Hispanic/Latin/Mexican    570249
Other                     442086
White                     413188
Black                     277243
Asian                      60637
Unknown                    54479
Name: Region_Ethnic_Origin, dtype: int64

In [24]:
df.to_csv('/content/sample_data/df_10_29Feb_4.csv', index=False)